In [5]:
# code block
import os
import inspect
import tkinter as tk
from tkinter import filedialog
import logging
import warnings
import ast
import openai
import callgpt
from callgpt import Chatbot

logger = logging.getLogger(__name__)

# Catch warnings
def fxn():
    warnings.warn("deprecated", DeprecationWarning)

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    fxn()

def open_file(filepath):
    with open(filepath, 'r', encoding='utf-8') as infile:
        return infile.read()

directory = filedialog.askdirectory()
os.chdir(directory)
warnings.filterwarnings("ignore")

def get_filenames_and_functions(dir_path):
    functions = {}
    for file in os.listdir(dir_path):
        file_path = os.path.join(dir_path, file)
        if file_path.endswith(".py"):
            filename = os.path.splitext(os.path.basename(file_path))[0]
            with open(file_path) as f:
                code = f.read()
            tree = compile(code, filename, "exec", ast.PyCF_ONLY_AST)
            functions[filename] = []
            for node in tree.body:
                if isinstance(node, ast.FunctionDef):
                    func_name = node.name
                    functions[filename].append(func_name)
    return functions


filenames_and_functions = get_filenames_and_functions(directory)
print(filenames_and_functions)

prompt = "Generate a summary of the codebase structure given the following filenames and functions:\n"+str(filenames_and_functions)+"\n\nSummary:"
chatbot = Chatbot()
summary = chatbot.smart_prompt(prompt)
print(summary)


2023-03-21 00:39:21.627 Python[54222:4595331] +[CATransaction synchronize] called within transaction


{'compliance_checker': ['open_file', 'generate_text', 'check_compliance'], 'compliance_checker_old': ['open_file', 'check_compliance'], 'database': ['init_db', 'insert_document', 'insert_message'], 'google_drive_old': ['get_drive_service', 'search_documents', 'generate_embeddings', 'get_all_documents', 'search_documents', 'get_document_text'], 'google_drive_WORKING': ['get_drive_service', 'get_document_text', 'build_index', 'get_embedding_model', 'search'], 'app': ['open_file', 'home', 'chat'], 'google_drive': ['get_drive_service', 'build_index'], 'slack_app': ['open_file', 'search_messages'], 'audit_logs': ['init_audit_logs_db', 'insert_audit_log'], 'app_WORKING': ['open_file', 'home', 'chat']}
User: Generate a summary of the codebase structure given the following filenames and functions:
{'compliance_checker': ['open_file', 'generate_text', 'check_compliance'], 'compliance_checker_old': ['open_file', 'check_compliance'], 'database': ['init_db', 'insert_document', 'insert_message'], '